# Basic Probability Models

To introduce the general notions of probability, the ALOHA Network Example will be used.

## ALOHA Network Example
Today's Ethernet evolved from an experimental network developed at the University of Hawaii, called ALOHA. A number of network nodes would occasionally try to use the same radio channel to communicate with a central computer. The nodes couldn't hear each other, due to the obstruction of mountains between them. If only one of them made an attempt to send, it would be successful, and it would receive an acknowledgement message in response from the central computer. But if more than one node were to transmit, a __collision__ would occur, garbling all the messages. The sending nodes would timeout after waiting for an acknowledgement which never came, and try
sending again later. To avoid having too many collisions, nodes would engage in random __backoff__, meaning that they would refrain from sending for a while even though they had something to send.

In [1]:
import numpy as np
import pandas as pd
import scipy.misc as misc
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
%matplotlib inline

In [2]:
class Sample:
    
    def __init__(self, ratio, num_samples = 1000):
        self.ratio = ratio
        self.num_samples = num_samples
        
    def generate(self):
        num_ones = int(self.ratio * self.num_samples)
        num_zeros = int((1 - self.ratio) * self.num_samples)
        
        return np.append(np.zeros(num_zeros), np.ones(num_ones))
    
    def event_happens(self):
        samples = self.generate()
        np.random.shuffle(samples)
        
        return samples[0] == 1

In [3]:
class InactiveNode:
    
    def __init__(self, p=None, q=None):
        self.p = p
        self.q = q
        
    def maybe_become_active(self):
        if Sample(self.q).event_happens():
            return ActiveNode(p = self.p, q = self.q)
        else:
            return InactiveNode(p = self.p, q = self.q)
        
    def is_sendable(self):
        return False
    
    def maybe_send(self, collision):
        return InactiveNode(p = self.p, q = self.q)
    
    def is_active(self):
        return False
    
    def is_inactive(self):
        return not self.is_active()

In [4]:
class ActiveNode:
    
    def __init__(self, p = None, q = None):
        self.sendable = False
        self.p = p
        self.q = q
        
    def maybe_become_active(self):
        return ActiveNode(p = self.p, q = self.q)
    
    def is_sendable(self):
        self.sendable = Sample(self.p).event_happens()
        
        return self.sendable
    
    def maybe_send(self, collision):
        if collision or not self.sendable:
            return ActiveNode(p = self.p, q = self.q)
        else:
            return InactiveNode(p = self.p, q = self.q)
        
    def is_active(self):
        return True
    
    def is_inactive(self):
        return not self.is_active()

In [5]:
class Epoch:
    
    def __init__(self, nodes, num):
        self.nodes = nodes
        self.num = num
        
    def check_collisions(self):
        collision = True
        
        for node in self.nodes:
            collision = collision & node.is_sendable()
        self.collision = collision
        
    def maybe_send(self):
        new_nodes = []
        for node in self.nodes:
            new_nodes.append(node.maybe_send(self.collision))
            
        return new_nodes
    
    def maybe_become_active(self):
        new_nodes = []
        for node in self.nodes:
            new_nodes.append(node.maybe_become_active())
            
        self.nodes = new_nodes
        
    def num_nodes_active(self):
        count = 0
        for node in self.nodes:
            if node.is_active():
                count += 1
        
        return count

In [6]:
class Criteria():
    
    def __init__(self, criteria=None, given=lambda x: True):
        self._criteria = criteria
        self._given = given
        
    def given(self, active_count):
        
        return self._given(active_count)
    
    def criteria(self, active_count):
        
        return self._criteria(active_count)

In [7]:
def simulate_aloha(criteria=None, 
                   given=lambda x: True, 
                   num_experiments=10000,
                   num_nodes=2,
                   num_epochs=2,
                   num_active_initial=2,
                   p=0.4,  # prob of sending as an active node
                   q=0.8): # prob of becoming active from inactive
    criteria = Criteria(criteria=criteria, given=given)
    
    def build_epoch(epoch=Epoch([ActiveNode(p=p, q=q),
                                 ActiveNode(p=p, q=q)], 0)):
        epoch.maybe_become_active()
        epoch.check_collisions()
        
        return Epoch(epoch.maybe_send(), epoch.num + 1)
    
    def generate_initial_number_of_nodes():
        nodes = []
        for i in range(num_active_initial):
            nodes.append(ActiveNode(p=p, q=q))
        for i in range(num_nodes - num_active_initial):
            nodes.append(InactiveNode(p=p, q=q))
        return nodes
    
    def populate_active_count(num_new_epochs):
        epoch = Epoch(generate_initial_number_of_nodes(), 0)
        active_count = [epoch.num_nodes_active()]
        for i in range(num_new_epochs):
            epoch = build_epoch(epoch=epoch)
            active_count.append(epoch.num_nodes_active())
        return active_count
    
    counts_updater = CountsUpdater(criteria)

    for experiment in range(num_experiments):
        active_count = populate_active_count(num_epochs)
        counts_updater.update(active_count)

    # needs to divide by num times that the "given" section 
    # happens, if "given" exists
    if counts_updater.denominator == 0:
        return 0.0
    else:
        return counts_updater.numerator / counts_updater.denominator

In [8]:
class CountsUpdater():
    
    def __init__(self, criteria):
        self.criteria = criteria
        self.numerator = 0.0
        self.denominator = 0.0
    
    def update(self, active_count):
        if self.criteria.given(active_count):
            self.denominator += 1
        if self.criteria.given(active_count) and \
           self.criteria.criteria(active_count):
            self.numerator += 1

In [9]:
simulate_aloha(criteria = lambda x: x[1] == 2)

0.5241

In [10]:
simulate_aloha(criteria=lambda x: x[1] == 1)

0.4837

In [11]:
simulate_aloha(criteria=lambda x: x[1] == 1,
               given = lambda x: x[2] == 2)

0.4198744769874477

In [12]:
simulate_aloha(criteria=lambda x: x[1] == 2 or x[2] == 2)

0.7173

In [13]:
simulate_aloha(lambda x: x[1] == 2 and x[2] == 1)

0.2455

In [14]:
simulate_aloha(lambda x: x[2]==0)

0.0391

In [15]:
simulate_aloha(criteria=lambda x: x[1]==1, given=lambda x: x[2]==1)

0.4923265807243708

In [16]:
def simulate_urn_drawing(criteria=None, given=None,
                         num_experiments=10000):
    draws = ['placeholder']
    experiments = []
    first_draws = []
    second_draws = []
    
    def urn_1(): 
        return ['B','B','B','Y','Y','Y']
    def urn_2():
        return ['B','B','B','B','B','Y','Y','Y','Y','Y','Y','Y']

    for i in range(num_experiments):
        first_draw = np.random.permutation(urn_1())[0]
        urn_2_copy = urn_2()
        urn_2_copy.append(first_draw)
        second_draw = np.random.permutation(urn_2_copy)[0]
        
        first_draws.append(first_draw)
        second_draws.append(second_draw)
        
    df = pd.DataFrame({'first_draw': first_draws,
                       'second_draw': second_draws})

    if given==None:
        return df[criteria(df)].shape[0] / float(num_experiments)
    else:
        meeting_what_is_given = df[given(df)]
        return meeting_what_is_given[criteria(meeting_what_is_given)].shape[0] \
                / float(meeting_what_is_given.shape[0])

In [17]:
simulate_urn_drawing(criteria=lambda df: df['second_draw']=='B')

0.4317

In [18]:
simulate_urn_drawing(criteria=lambda df: df['first_draw']=='B',
                     given=lambda df: df['second_draw']=='B')

0.5419664268585132

In [19]:
def bug_percentage(num_programs=5,
                   exactly_have_bug=2, 
                   num_experiments=10000, 
                   prob_bug=0.2):
    ones = np.ones(int(prob_bug * 10))
    zeros = np.zeros(int((1.0-prob_bug) * 10))
    
    vals = np.append(ones, zeros)
   
    count = 0
    for i in range(num_experiments):
        
        result = []
        for x in range(num_programs):
            val = np.random.permutation(vals)[0]
            result.append(val)
            
        if np.array(result).sum() == exactly_have_bug:
            count += 1
    
    return count / float(num_experiments)
        
bug_percentage()

0.206